In [1]:
!pip install datasets

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 480.6/480.6 kB 13.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 7.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.3/179.3 kB 10.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 7.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 8.6 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2024.10.0
    Uninstalling fsspec-2024.10.0:
      Successfully uninstalled fsspec-2024.10.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2024.10.0 requires fsspec==2024.10.0, but you have fsspec 2024.9.0 which is incompatible.


In [2]:
import random
from datasets import DatasetDict
from datasets import load_dataset

duc_dataset = load_dataset("midas/duc2001")

random.seed(42)

shuffled_dataset = duc_dataset["test"].shuffle(seed=42)

train_ratio = 0.8
train_size = int(len(shuffled_dataset) * train_ratio)

train_dataset = shuffled_dataset.select(range(train_size))
test_dataset = shuffled_dataset.select(range(train_size, len(shuffled_dataset)))

duc_split = DatasetDict({
    "train": train_dataset,
    "test": test_dataset
})

duc_split

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


README.md:   0%|          | 0.00/4.32k [00:00<?, ?B/s]

duc2001.py:   0%|          | 0.00/5.58k [00:00<?, ?B/s]

The repository for midas/duc2001 contains custom code which must be executed to correctly load the dataset. You can inspect the repository content at https://hf.co/datasets/midas/duc2001.
You can avoid this prompt in future by passing the argument `trust_remote_code=True`.

Do you wish to run the custom code? [y/N] y


Repo card metadata block was not found. Setting CardData to empty.


test.jsonl:   0%|          | 0.00/3.62M [00:00<?, ?B/s]

Generating test split: 0 examples [00:00, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['id', 'document', 'doc_bio_tags'],
        num_rows: 246
    })
    test: Dataset({
        features: ['id', 'document', 'doc_bio_tags'],
        num_rows: 62
    })
})

In [3]:
import os
import numpy as np
from transformers import Trainer, TrainingArguments, BertForTokenClassification, BertTokenizerFast
from nltk.stem import PorterStemmer
from sklearn.metrics import f1_score

inspec_dataset = load_dataset("midas/inspec")

tokenizer = BertTokenizerFast.from_pretrained("bert-base-uncased")

label_to_id = {"O": 0, "B": 1, "I": 2}
id_to_label = {v: k for k, v in label_to_id.items()}

def preprocess_data(dataset):
    def tokenize_and_align_labels(examples):
        tokenized_inputs = tokenizer(
            examples["document"],
            truncation=True,
            padding="max_length",
            max_length=512,
            is_split_into_words=True
        )
        labels = []
        for i, label in enumerate(examples["doc_bio_tags"]):
            word_ids = tokenized_inputs.word_ids(batch_index=i)
            label_ids = []
            previous_word_idx = None
            for word_idx in word_ids:
                if word_idx is None:
                    label_ids.append(-100)
                elif word_idx != previous_word_idx:
                    label_ids.append(label_to_id[label[word_idx]])
                else:
                    label_ids.append(-100)
                previous_word_idx = word_idx
            labels.append(label_ids)
        tokenized_inputs["labels"] = labels
        return tokenized_inputs

    return dataset.map(tokenize_and_align_labels, batched=True)

tokenized_duc = preprocess_data(duc_split)
tokenized_inspec = preprocess_data(inspec_dataset)

The cache for model files in Transformers v4.22.0 has been updated. Migrating your old cache. This is a one-time only operation. You can interrupt this and resume the migration later on by calling `transformers.utils.move_cache()`.


0it [00:00, ?it/s]

README.md:   0%|          | 0.00/25.4k [00:00<?, ?B/s]

inspec.py:   0%|          | 0.00/6.47k [00:00<?, ?B/s]

The repository for midas/inspec contains custom code which must be executed to correctly load the dataset. You can inspect the repository content at https://hf.co/datasets/midas/inspec.
You can avoid this prompt in future by passing the argument `trust_remote_code=True`.

Do you wish to run the custom code? [y/N] y


...

Map:   0%|          | 0/246 [00:00<?, ? examples/s]

Map:   0%|          | 0/62 [00:00<?, ? examples/s]

Map:   0%|          | 0/1000 [00:00<?, ? examples/s]

Map:   0%|          | 0/500 [00:00<?, ? examples/s]

Map:   0%|          | 0/500 [00:00<?, ? examples/s]

In [4]:
import os
import numpy as np
from transformers import Trainer, TrainingArguments, BertForTokenClassification, BertTokenizerFast
from nltk.stem import PorterStemmer
from sklearn.metrics import f1_score

inspec_dataset = load_dataset("midas/inspec")

tokenizer = BertTokenizerFast.from_pretrained("bert-base-uncased")

label_to_id = {"O": 0, "B": 1, "I": 2}
id_to_label = {v: k for k, v in label_to_id.items()}

def preprocess_duc(dataset):
    return tokenizer(dataset["document"], truncation=True, padding="max_length", max_length=512, is_split_into_words=True)

def preprocess_inspec(dataset):
    def tokenize_and_align_labels(examples):
        tokenized_inputs = tokenizer(
            examples["document"],
            truncation=True,
            padding="max_length",
            max_length=512,
            is_split_into_words=True
        )
        labels = []
        for i, label in enumerate(examples["doc_bio_tags"]):
            word_ids = tokenized_inputs.word_ids(batch_index=i)
            label_ids = []
            previous_word_idx = None
            for word_idx in word_ids:
                if word_idx is None:
                    label_ids.append(-100)
                elif word_idx != previous_word_idx:
                    label_ids.append(label_to_id[label[word_idx]])
                else:
                    label_ids.append(-100)
                previous_word_idx = word_idx
            labels.append(label_ids)
        tokenized_inputs["labels"] = labels
        return tokenized_inputs

    return dataset.map(tokenize_and_align_labels, batched=True)

tokenized_duc = duc_split.map(preprocess_duc, batched=True)
tokenized_inspec = preprocess_inspec(inspec_dataset)

Repo card metadata block was not found. Setting CardData to empty.


Map:   0%|          | 0/246 [00:00<?, ? examples/s]

Map:   0%|          | 0/62 [00:00<?, ? examples/s]

Map:   0%|          | 0/1000 [00:00<?, ? examples/s]

Map:   0%|          | 0/500 [00:00<?, ? examples/s]

Map:   0%|          | 0/500 [00:00<?, ? examples/s]

In [5]:
from transformers import BertForMaskedLM

model = BertForMaskedLM.from_pretrained("bert-base-uncased")
model.to("cuda")


model.safetensors:   0%|          | 0.00/440M [00:00<?, ?B/s]

BertForMaskedLM has generative capabilities, as `prepare_inputs_for_generation` is explicitly overwritten. However, it doesn't directly inherit from `GenerationMixin`. From 👉v4.50👈 onwards, `PreTrainedModel` will NOT inherit from `GenerationMixin`, and this model will lose the ability to call `generate` and other related functions.
  - If you're using `trust_remote_code=True`, you can get rid of this warning by loading the model with an auto class. See https://huggingface.co/docs/transformers/en/model_doc/auto#auto-classes
  - If you are the owner of the model architecture code, please modify your model class such that it inherits from `GenerationMixin` (after `PreTrainedModel`, otherwise you'll get an exception).
  - If you are not the owner of the model architecture class, please contact the model code owner to update it.
Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForMaskedLM: ['bert.pooler.dense.bias', 'bert.pooler.dense.weight', 'c

BertForMaskedLM(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0-11): 12 x BertLayer(
          (attention): BertAttention(
            (self): BertSdpaSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12, elementwi

In [8]:
from transformers import Trainer, TrainingArguments, DataCollatorForLanguageModeling, BertForMaskedLM
import os
os.environ['WANDB_DISABLED'] = 'true'

data_collator = DataCollatorForLanguageModeling(
    tokenizer=tokenizer,
    mlm=True,
    mlm_probability=0.15,
)

training_args = TrainingArguments(
    output_dir="./results/duc_mlm",
    evaluation_strategy="epoch",
    learning_rate=5e-5,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    num_train_epochs=3,
    weight_decay=0.01,
    save_total_limit=2,
    logging_dir="./logs",
    save_strategy="epoch",
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_duc["train"],
    eval_dataset=tokenized_duc["test"],
    data_collator=data_collator,
    tokenizer=tokenizer,
)

trainer.train()

trainer.save_model("./duc_mlm_model")

/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1568: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(
Using the `WANDB_DISABLED` environment variable is deprecated and will be removed in v5. Use the --report_to flag to control the integrations used for logging result (for instance --report_to none).
<ipython-input-8-9c7b934b62ce>:24: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


Epoch,Training Loss,Validation Loss
1,No log,1.978553
2,No log,1.827643
3,No log,1.892213


In [9]:
import os
os.environ['WANDB_DISABLED'] = 'true'

In [16]:
from transformers import Trainer, TrainingArguments, BertForTokenClassification
import numpy as np
from sklearn.metrics import f1_score, precision_score, recall_score, accuracy_score

def compute_metrics(pred):
    logits, labels = pred
    predictions = np.argmax(logits, axis=-1)

    true_labels = [
        [label for label in label_row if label != -100]
        for label_row in labels
    ]
    true_predictions = [
        [pred for pred, label in zip(pred_row, label_row) if label != -100]
        for pred_row, label_row in zip(predictions, labels)
    ]

    flat_true_labels = [item for sublist in true_labels for item in sublist]
    flat_true_predictions = [item for sublist in true_predictions for item in sublist]

    accuracy = accuracy_score(flat_true_labels, flat_true_predictions)
    precision = precision_score(flat_true_labels, flat_true_predictions, average="weighted")
    recall = recall_score(flat_true_labels, flat_true_predictions, average="weighted")
    f1 = f1_score(flat_true_labels, flat_true_predictions, average="weighted")

    return {
        "accuracy": accuracy,
        "precision": precision,
        "recall": recall,
        "f1": f1,
    }

model = BertForTokenClassification.from_pretrained(
    "./duc_mlm_model",
    num_labels=len(label_to_id),
)

inspec_training_args = TrainingArguments(
    output_dir="./results/inspec_finetuning",
    evaluation_strategy="epoch",
    learning_rate=5e-5,
    per_device_train_batch_size=16,
    num_train_epochs=5,
    weight_decay=0.01,
    logging_dir="./logs",
    save_total_limit=2,
    load_best_model_at_end=True,
    save_strategy="epoch",
)

inspec_trainer = Trainer(
    model=model,
    args=inspec_training_args,
    train_dataset=tokenized_inspec["train"],
    eval_dataset=tokenized_inspec["validation"],
    tokenizer=tokenizer,
    compute_metrics=compute_metrics,
)

inspec_trainer.train()

results = inspec_trainer.evaluate(tokenized_inspec["test"])
results


Some weights of BertForTokenClassification were not initialized from the model checkpoint at ./duc_mlm_model and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1568: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(
Using the `WANDB_DISABLED` environment variable is deprecated and will be removed in v5. Use the --report_to flag to control the integrations used for logging result (for instance --report_to none).
<ipython-input-16-96141c459292>:51: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  inspec_trainer = Trainer(


Epoch,Training Loss,Validation Loss,Accuracy,Precision,Recall,F1
1,No log,0.205244,0.902611,0.898093,0.902611,0.898913
2,No log,0.195934,0.908796,0.908147,0.908796,0.908208
3,No log,0.202447,0.906156,0.914345,0.906156,0.909445
4,No log,0.215537,0.907001,0.908882,0.907001,0.907832
5,No log,0.225735,0.907710,0.910510,0.907710,0.908910


{'eval_loss': 0.19597378373146057,
 'eval_accuracy': 0.9080980683506686,
 'eval_precision': 0.9050980959794315,
 'eval_recall': 0.9080980683506686,
 'eval_f1': 0.9062223948150577,
 'eval_runtime': 15.6772,
 'eval_samples_per_second': 31.893,
 'eval_steps_per_second': 4.019,
 'epoch': 5.0}

{'eval_loss': 0.19597378373146057,
 'eval_accuracy': 0.9080980683506686,
 'eval_precision': 0.9050980959794315,
 'eval_recall': 0.9080980683506686,
 'eval_f1': 0.9062223948150577,
 'eval_runtime': 15.6772,
 'eval_samples_per_second': 31.893,
 'eval_steps_per_second': 4.019,
 'epoch': 5.0}

In [11]:
import numpy as np
from sklearn.metrics import f1_score
from transformers import Trainer, TrainingArguments

eval_args = TrainingArguments(
    output_dir="./results",
    per_device_eval_batch_size=8,
    logging_dir="./logs",
)

trainer = Trainer(
    model=model,
    args=eval_args,
    eval_dataset=tokenized_duc["test"],
    tokenizer=tokenizer,
    compute_metrics=compute_metrics,
)

results = trainer.evaluate()
results

Using the `WANDB_DISABLED` environment variable is deprecated and will be removed in v5. Use the --report_to flag to control the integrations used for logging result (for instance --report_to none).
<ipython-input-11-7c8bc08c8c36>:11: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


{'eval_model_preparation_time': 0.0032,
 'eval_runtime': 1.991,
 'eval_samples_per_second': 31.14,
 'eval_steps_per_second': 4.018}

In [18]:
results

{'eval_loss': 0.19597378373146057,
 'eval_accuracy': 0.9080980683506686,
 'eval_precision': 0.9050980959794315,
 'eval_recall': 0.9080980683506686,
 'eval_f1': 0.9062223948150577,
 'eval_runtime': 15.6772,
 'eval_samples_per_second': 31.893,
 'eval_steps_per_second': 4.019,
 'epoch': 5.0}

In [12]:
model.save_pretrained("./inspec_transfer_model")
tokenizer.save_pretrained("./inspec_transfer_model")

('./inspec_transfer_model/tokenizer_config.json',
 './inspec_transfer_model/special_tokens_map.json',
 './inspec_transfer_model/vocab.txt',
 './inspec_transfer_model/added_tokens.json',
 './inspec_transfer_model/tokenizer.json')

In [13]:
!zip -r inspec_transfer_model.zip inspec_transfer_model

  adding: inspec_transfer_model/ (stored 0%)
  adding: inspec_transfer_model/vocab.txt (deflated 53%)
  adding: inspec_transfer_model/config.json (deflated 50%)
  adding: inspec_transfer_model/tokenizer_config.json (deflated 76%)
  adding: inspec_transfer_model/tokenizer.json (deflated 71%)
  adding: inspec_transfer_model/special_tokens_map.json (deflated 42%)
  adding: inspec_transfer_model/model.safetensors (deflated 7%)


In [ ]:
import tqdm
import torch

all_keyphrases = []
id_to_label = {0: "O", 1: "B", 2: "I"}


def extract_keyphrases(tokens, labels):
    keyphrases = []
    current_phrase = []

    for idx, (token, label) in enumerate(zip(tokens, labels)):
        if label == "B":
            if current_phrase:
                keyphrases.append(
                    " ".join(current_phrase),
                )
            current_phrase = [token]
            start_position = idx
        elif label == "I" and current_phrase:
            current_phrase.append(token)
        else:
            if current_phrase:
                keyphrases.append({
                    "phrase": " ".join(current_phrase),
                })
                current_phrase = []
                start_position = None

    if current_phrase:
        keyphrases.append({
            "phrase": " ".join(current_phrase),
        })

    return keyphrases

for example in tqdm.tqdm(tokenized_duc["test"]):
    input_ids = example['input_ids']
    attention_mask = example['attention_mask']
    labels = example['labels']

    true_labels = [id_to_label[label] if label != -100 else "O" for label in labels]

    model.eval()
    with torch.no_grad():
        outputs = model(input_ids=torch.tensor([input_ids]), attention_mask=torch.tensor([attention_mask]))
        predictions = torch.argmax(outputs.logits, dim=-1).squeeze().tolist()

    predicted_labels = [id_to_label[pred] for pred in predictions]
    tokens = tokenizer.convert_ids_to_tokens(input_ids)
    tokens = tokens[1:-1]  # Remove [CLS] and [SEP]
    predicted_labels = predicted_labels[1:-1]

    keyphrases = extract_keyphrases(tokens, predicted_labels, tokenizer)
    all_keyphrases.append({
        "document_id": example['id'],
        "document": example['document'],
        "true_labels": true_labels,
        "predicted_labels": predicted_labels,
        "keyphrases": keyphrases
    })

print(f"Total documents processed: {len(all_keyphrases)}")

In [15]:
tokenized_duc["test"][0].keys()

dict_keys(['id', 'document', 'doc_bio_tags', 'input_ids', 'token_type_ids', 'attention_mask'])

In [ ]:
for doc in tokenized_duc["test"]:
    doc['labels']